In [1]:
import os
import random
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from utils import *
import warnings
warnings.filterwarnings('ignore')


NUM_GENE = 100 #number of genes to be used
# get all_genes: all genes appeared in all the ontologies
bf = pd.read_csv('input_data/hgncToKEGG.csv')
KEGG_genes = dict(zip(bf['symbol'], bf['path'] ))
bf = pd.read_csv('input_data/hgncToHPO.csv')
HPO_genes = dict(zip(bf['gene'], bf['pathway'] ))
bf = pd.read_csv('input_data/hgncToDO.csv')
DO_genes = dict(zip(bf['gene'], bf['pathway'] ))
all_genes = list(set(KEGG_genes.keys()).intersection(set(HPO_genes.keys())).intersection(set(DO_genes.keys()))) 

# get gene_go_all: dictionary of gene - KEGG_ontology for all genes appeared in KEGG 
# we could use other ontologies here, e.g., HPO or DO, instead of KEGG
df = pd.read_csv('input_data/hgncToKEGG.csv')
gene_go_all = dict(zip(df['symbol'], df['path'] ))
df = pd.read_csv('input_data/UK207.csv')
# filter out data points not having the label
df = df[ df['event'].notnull()]
df = df.dropna(axis=1, how='any')
# only consider genes appeared in all the three ontologies, making them comparable
genes = list(set(all_genes).intersection(set(df.columns)))
sel = VarianceThreshold()
sel.fit(df[genes].values)
score = sel.variances_
# selecting NUM_GENE genes, stored in top_genes
idx = np.argsort(score, 0)[::-1][:NUM_GENE]
top_genes = [ genes[e] for e in idx ]
print('len(top_genes):', len(top_genes))
with open('output_data/top_genes.txt','w') as f:
    f.write('\n'.join(top_genes))

# get gene_go: dictionary of gene - KEGG_ontology for the top genes
# and its reverse, go_gene, a dictionary of KEGG_ontology - gene
gene_go = {}
for gene in gene_go_all:
    if gene in top_genes:
        gene_go[gene] = gene_go_all[gene]
go_gene = {}
for gene in gene_go:
    goes = gene_go[gene].split('|')
    for go in goes:
        try:
            go_gene[go] += [gene]
        except:
            go_gene[go] = [gene]

# get gene expression and the label for top genes and store it in gene_condition
gene_condition = df[ top_genes + ['event'] ]
gene_condition['event'] = gene_condition['event'].apply(int)
gene_condition.columns = top_genes + [ 'condition' ]
gene_condition.to_csv('output_data/gene_condition.csv',index=None)
# store X and y separately
non_graph = gene_condition[ top_genes ]
non_graph.to_csv('output_data/non_graph.csv',index=None,header=None)
yy = gene_condition['condition'].values.astype(int)
with open('output_data/y.txt','w') as f:
    f.write('\n'.join(map(str,yy)))
# gene expression: will be used as input for LIONESS    
exp = non_graph.T
exp.to_csv('output_data/gene_expression.txt',header=None,sep='\t')    

# create ten-fold splits to be used for all models
for seed in range(10):
    idx_0 = np.where( yy==0 )[0].flatten()
    idx_1 = np.where( yy==1 )[0].flatten()
    random.Random(seed).shuffle(idx_0)
    idx_0 = idx_0[:len(idx_1)]
    idx = list(idx_0) + list(idx_1)
    random.Random(seed).shuffle(idx)
    y = yy[idx]
    X_train, X_test, y_train, y_test = train_test_split(idx, y,test_size=0.3,random_state=0,stratify=y)
    directory = '10fold_idx' 
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(directory + '/train_idx-' + str(seed+1) + '.txt', 'w') as f:
        f.write('\n'.join(map(str,X_train)))
    with open(directory + '/test_idx-' + str(seed+1) + '.txt', 'w') as f:
        f.write('\n'.join(map(str,X_test)))


# create PAN graphs for non_graph data where the link is built from the relation of KEGG ontologies among selected genes
gene_list = list(gene_go.keys())
gene_list.sort()
gene_idx = {gene_list[i]:i for i in range(len(gene_list))}
go_list = []
for e in gene_go:
    go_list += gene_go[e].split('|')
go_list = list(set(go_list))
go_list.sort()
go_idx = {go_list[i]:i for i in range(len(go_list))}

M = np.zeros((len(gene_idx),len(go_idx)))
print('M.shape:', M.shape)
for gene in gene_idx:
    g_idx = gene_idx[gene]
    goes = gene_go[gene].split('|')
    for go in goes:
        M[g_idx][go_idx[go]] = 1        

NUM_COL = len(go_idx)
exp = non_graph[gene_list].values
# create continuous graph
with open('output_data/pan_graph.csv','w') as f:
    ls = [] 
    for i in range(NUM_COL-1):
        for j in range(i+1,NUM_COL):
            ls += [ str(i) + '_' + str(j) ]
    f.write(','.join(ls) + '\n')
    count = 0
    for e in exp:
        ls = [] 
        m = np.multiply(M,np.reshape(e, (len(e),1)))
        m = m.T
        D = scipy.spatial.distance.cdist(m,m)
        for i in range(NUM_COL-1):
            for j in range(i+1,NUM_COL):
                dst = D[i,j]
                if dst==0:
                    dst = 1e-6
                ls += [ round(1/dst,4)  ]
        f.write(','.join(map(str,ls)) + '\n') 
        
NUM_EDGE = 200 # number of edges for adjacency graph
# turn graphs with continous weights to adjacency graph
GG = to_adjacency_G('output_data/pan_graph.csv',NUM_EDGE)
# get features for this adjacency graph
feats = get_closeness_centrality(GG)
# store the feature in 'pan_graph_feature.csv'
with open('output_data/pan_graph_feature.csv', 'w') as f:
    for e in feats:
        f.write(','.join(map(str,e)) + '\n')        
        
print('Done creating PAN graph!')         

len(top_genes): 100
M.shape: (100, 51)
Number of nodes: 51
Done creating PAN graph!
